In [1]:
#This versions creates visitor nodes and cbg nodes. 
import pandas as pd #importing neccisary libraries 
import numpy as np
import ast
import datetime
import calendar

/Users/eho/.pyenv/versions/3.8.10/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [15]:
%%bash
which python3
python3 -m pip install neo4j

/Users/eho/.pyenv/versions/3.8.10/bin/python3


In [16]:
# you might have to change the uri to whatever pops up when you run neo
#connect to neo4j database 
from neo4j import GraphDatabase 
driver = GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", "test")) 
session = driver.session()

In [17]:
df = pd.read_csv('../data/safegraph/jan.csv') # read csv to pandas

In [18]:
#isolate specified colums will be changing them to more meanningful names
miamiv = df.visits_by_day 
miamivv = df.visitor_home_cbgs

In [19]:
# Will add another part to have all the csv be read at once so I do not have to keep loading it.
#This for loop creates and loads all nodes and edges

#len(df)
#this for loop I am createing all the places and cities and zips
for i in range(len(df)):
    #setting variables that I will use to create the nodes. 
    pk= df.placekey[i]
    name = df.location_name[i]
    zipc= df.postal_code[i]
    address= df.street_address[i]
    lat= df.latitude[i]
    long= df.longitude[i]
    topCat= df.top_category[i]
    subCat= df.sub_category[i]
    city= df.city[i]
    poicbg = df.poi_cbg[i]
    
    #These are the queries that I will pass to neo4j that will create all the place, city and zip nodes    
    q1='MERGE(p:Place{name:'+'"'+name+'"' +', city:'+'"' +city+'"'+', zipcode:'+"'" +str(zipc)+"'" +', placeKey:'+"'" +pk+"'" +', address:'+'"' +address+'"' +', topCatagory:'+'"'+str(topCat)+'"'+', subCatagory:'+'"'+str(subCat)+'"' +', cbg: '+"'" +str(poicbg) +"'" +', location:point({x:toFloat('+str(lat)+'),y:toFloat('+str(long)+')})})'
    q2='MERGE(z:Zip{name: '+"'" +str(zipc)+"'" +', zipC:'+"'"+str(zipc)+"'"+', city:'+"'" +city+"'"+'});'
    q3='MERGE(c:City{name: '+'"' +city+'"'+',  city:'+"'" +city+"'"+'});'
    q4='MATCH (p:Place), (z:Zip) WHERE p.name = '+'"' +name +'"' + 'AND p.city = '+"'" + city +"'"+ 'AND z.city = '+"'" +city+"'" + 'AND p.zipcode = '+"'" +str(zipc)+"'" +' AND z.name = '+"'" +str(zipc)+"'" +' MERGE(p)-[r:IS_WITHIN]->(z);'
    q5='MATCH (c:City), (z:Zip) WHERE c.name = '+'"' +city +'"' +' AND z.city = '+"'" +city+"'" + ' AND z.name = '+"'" +str(zipc)+"'" +' MERGE(z)-[r:IS_WITHIN]->(c);'
    #Match staments are the edges that connect the two nodes
    
    #running the querries 
    session.run(q1)
    session.run(q2)
    session.run(q3)
    session.run(q4)
    session.run(q5)
    #print(q5)
    
  
    
    
     # miamiv =  df.visits_by_day 
    
    #string ->dictionary-> nparray
    
    if((type(miamiv[i]) == float) == True):
        itt= 0
    
    else:
        miami3 = miamiv[i]
        testnp = ast.literal_eval(miami3)
        
        itt = len(testnp)
                
        #test=ast.literal_eval(miamiv[i])
        #test2 = list(test.items())
        #testnp = np.array(test2)
        #itt = len(testnp)
        
    
        dates= df.date_range_start[i]
    
        #parsing the date month and year
        year=dates[:4]
        month=dates[5:7]
        day=dates[8:10]
    
    if itt != 0:
        #range(itt)
        for x in range(itt):
            #here I am creating visitor nodes 
            if (testnp[x] == 0):
                nothing =""
            else: 
                
                ndate = int(day) +x
                visits = testnp[x] 
                date = ""+str(month)+"/"+str(ndate)+"/"+str(year)+""
                monthName = calendar.month_name[int(month)]
                weekDay= datetime.date(int(year),int(month),(int(day)+x)).strftime('%A')
            
                qqq1= 'MERGE(v:Visitor{month:'+"'" +str(monthName)+"'" +', day:'+str(ndate)+', year:'+str(year)+', weekDay:'+"'" +str(weekDay)+"'" +', date:'+"'" +str(date)+"'" +', size:'+str(visits)+', placeKey:'+"'" +str(pk) +"'" +' })'
                qqq2 ='MATCH (p:Place), (v:Visitor) WHERE p.placeKey = '+"'" +pk +"'" + ' AND v.date = '+"'" + str(date)+"'"   +' AND v.placeKey = '+"'" +pk +"'" +' MERGE(v)-[r:VISITED]->(p)'
                #print(qq1)
            
                session.run(qqq1) #remeber to change the 0 to i
                session.run(qqq2)
                #print(qq1)
                
                
                
         # miamivv =  df.visitor_home_cbgs
        #json ->dictionary-> nparray
        if((type(miamivv[i]) == float) == True):
             iti= 0
    
        else:
            test=ast.literal_eval(miamivv[i])
            test2 = list(test.items())
            testnpp = np.array(test2)
            iti = len(testnpp)
        
       
        if iti != 0:
            
            da=0
            dasum=0
            percentage = []
            for x in range(iti):
                
                da = int(testnpp[x][1])   
                dasum += da # sum 
                percentage =[]

            for j in range(len(testnpp)):
                #here I am createing Census block nodes and connecting them with the places they have been. 
                odds = (float((testnpp[j][1]))/dasum)  
                percentage += [odds]
                p1= (str(float(percentage[j] *100))+'%')

                node= testnpp[j][0] # this is the keys
                prop= testnpp[j][1] # this is the values
                
                state = node[:2]
                county = node[2:5]
                tract = node[5:11]
                blockGroup =node[11]
                
                qq1= 'MERGE(g:CensusBG{cbg:'+"'" +node+"'"+ ', State:'+"'"+state+"'"+' , County:'+"'"+county+"'"+', Tract:'+"'"+tract+"'"+', BlockGroup:'+"'"+blockGroup+"'"+' })'
                qq2 ='MATCH (p:Place), (g:CensusBG) WHERE p.placeKey = '+"'" +pk +"'"  +' AND g.cbg = '+"'" +node+"'" +' MERGE(g)-[b:Visited{ PercentofVisits:'+"'"+p1+"'"+'}]->(p)'
                session.run(qq1) #remeber to change the 0 to i
                session.run(qq2)
            #print(qq2)

IndexError: string index out of range

In [ ]:
#Also if you stop the kernel you will have to re run the parts where you connect to neo